In [1]:
#Set-up
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import warnings
from IPython.display import clear_output
import nibabel as nib

import torch
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.nn.functional as F
import time
import copy

In [2]:
regression = True

In [3]:
# df = pd.read_csv('./Exploration/ixi_demographics.csv')

#Collect all pre-processed images from .anat directories
skullstripped_img_list = []
ori_img_list = []
seg_img_list = []
file_id = []
sample_size = 0
root_path = "/shared/ixi-dataset/"
for i in range(19):
    dir_name = root_path + "IXI-T1-" + str(i)
    anat_directories = [x for x in os.listdir(dir_name) if x.endswith('.anat')]
    for anat_directory in anat_directories:
        ori_file_name = dir_name + '/' + anat_directory + '/T1_biascorr.nii.gz'
        seg_file_name = dir_name + '/' + anat_directory + "/T1_fast_seg.nii.gz"
        if anat_directory[0:3] == 'IXI' and os.path.exists(ori_file_name) and os.path.exists(seg_file_name):
            ori_img = nib.load(ori_file_name).get_data()
            seg_img = nib.load(seg_file_name).get_data()
            skullstripped_img = ori_img * (seg_img > 0)
            ori_img_list.append(ori_img)
            seg_img_list.append(seg_img)
            skullstripped_img = (skullstripped_img - np.mean(skullstripped_img)) / np.std(skullstripped_img)
            skullstripped_img_list.append(skullstripped_img)
            file_id.append(anat_directory[3:6])
            sample_size += 1
            clear_output(wait=True)
            print(sample_size)

print(len(skullstripped_img_list))
n = len(skullstripped_img_list)

536
536


In [4]:
new_img_list = []

for i in range(len(skullstripped_img_list)):    
    new_img = skullstripped_img_list[i][16:240,90,:]
    #modify the dimensions by cropping and zero-padding so that all the images' sizes become 224*224
    d_2 = new_img.shape[1]
    if d_2 % 2 == 0:
        pad_width = int((224 - d_2) / 2)
        new_img = np.pad(np.asarray(new_img),((0,0),(pad_width, pad_width)),"constant") 
    else: 
        pad_width_0 = int((224 - d_2) / 2)
        pad_width_1 = pad_width_0 + 1
        new_img = np.pad(np.asarray(new_img),((0,0),(pad_width_0, pad_width_1)),"constant") 
    final_img = np.tile(new_img,(3,1)).reshape((3,224,224))
    new_img_list.append(final_img) 

x = np.asarray(new_img_list)
x = torch.Tensor(x)
torch.save(x, "data.pt")

In [5]:
df = pd.read_csv('./demographics.csv')
sheet = df.values
filtered = list()
for i in range(0, len(sheet)):
    if sheet[i, 5][0:3] == 'IXI':
         filtered.append((sheet[i, 5], sheet[i, 4]))
sheet = filtered

#Create the labels for the skullstripped images
age = list()
for i in range(0, len(file_id)):
    for patient in sheet:
        if patient[0][3:6] == file_id[i]:
            age.append(patient[1])

if regression == True:
    y = np.asarray(age)
    y = y.reshape((y.shape[0], 1))
else:  
    target_age = list()
    for i in range(0, len(age)):
        current_age = int(round(age[i]))
        current_age -= 18
        if current_age >= 60:
            target_age.append(12)
        else:
            target_age.append(current_age // 5)
    y = np.asarray(target_age)

y = torch.Tensor(y)
torch.save(y, "labels.pt")